In [1]:
from data_clean import get_int
import pandas as pd
import numpy as np
import json


In [2]:
def rewards_features(df_rewards):
    import re
    df_rewards.pledges = df_rewards.pledges.apply(json.loads)
    df_rewards['num_rewards'] = df_rewards.pledges.apply(len)

    df_rewards['num_with_limits'] = df_rewards.pledges.apply(lambda x: len([p['limit'] for p in x if (p['limit']!='NULL' and p['limit']!='Reward no longer available')]))
    df_rewards['num_with_limits_pcnt'] = df_rewards.apply(lambda x: float(x['num_with_limits'])/ x['num_rewards'] if x['num_rewards'] else 0, axis=1)

    df_rewards['moneys'] = df_rewards.pledges.apply(lambda x: np.array([get_int(re.findall(r'[-+]?\d+(?:,\d+)?[\.]?\d*',p['money'])[0]) for p in x]))

    df_rewards['money_max'] = df_rewards.moneys.apply(lambda x: x.max() if len(x) else 0)
    df_rewards['money_min'] = df_rewards.moneys.apply(lambda x: x.min() if len(x) else 0)
    df_rewards['money_median'] = df_rewards.moneys.apply(lambda x: np.median(x) if len(x) else 0)
    df_rewards['money_std'] = df_rewards.moneys.apply(lambda x: np.std(x) if len(x) else 0)

    df_rewards['money_max_pcnt'] = df_rewards.apply(lambda x: x.money_max*1.0/x.goal, axis=1)
    df_rewards['money_min_pcnt'] = df_rewards.apply(lambda x: x.money_min*1.0/x.goal, axis=1)
    df_rewards['money_median_pcnt'] = df_rewards.apply(lambda x: x.money_median*1.0/x.goal, axis=1)
    df_rewards['money_std_pcnt'] = df_rewards.apply(lambda x: x.money_std*1.0/x.goal, axis=1)
    return df_rewards.drop(['goal','pledges', 'moneys'], axis=1)

In [6]:
df.category_parent.value_counts()

music           24119
publishing      19175
film & video    18571
art             14182
food             9859
technology       8137
games            6458
design           5882
fashion          5702
photography      3800
crafts           3024
theater          1332
comics           1175
dance            1135
journalism         40
dtype: int64

In [2]:
df = pd.read_csv('data/projects_features.csv')
df_rewards = pd.read_csv('data/projects_rewards.csv')

In [4]:
df_loc = pd.read_csv('data/projects_location.csv')
df_disposable_income = pd.read_csv('data/Disposible income.csv')

In [5]:
disposable_income = df_disposable_income.merge(
    df_loc[['id', 'proj_location_display_name']], 
    left_on='City', right_on=['proj_location_display_name'], 
    how='right')[['id', 'Income']]

In [6]:
rewards_ftr = rewards_features(df_rewards[~df_rewards.pledges.isnull()].merge(df[['id','goal']]))

In [7]:
df = df.merge(rewards_ftr, how='left').merge(disposable_income, how='left')

In [9]:
drop_cols = ['goal', 'pledged', 'static_usd_rate', 'usd_pledged', 'category_name', 'spotlight', 
             'has_community', 'has_creator-bio', 'has_creator-full', 'has_proj', 'user_deleted']

11


In [10]:
df = df.drop(drop_cols, axis=1).set_index('id')

In [11]:
trainset = df.sample(frac=0.8)

In [12]:
testset = df[~df.index.isin(trainset.index)]

In [13]:
(set(testset.index)|set(trainset.index)) == set(df.index)

True

In [14]:
df.to_csv('data/kick_all.csv')
trainset.to_csv('data/kick_training_set.csv')
testset.to_csv('data/kick_testing_set.csv')

In [16]:
def distribution(df):
    size = df.shape[0]
    category = df.category_parent.value_counts()/size
    category.index = ['category_'+x for x in category.index]
    state = df.state.value_counts()/size
    state.index = ['state_'+x for x in state.index]
    goal_mag = df.goal_magnitude.value_counts()/size
    goal_mag.index = ['goal_magnitude_'+str(x) for x in goal_mag.index]
    return category.append(state).append(goal_mag)

In [17]:
train_category = distribution(trainset)
all_category = distribution(df)
test_category = distribution(testset)

In [19]:
pd.DataFrame([all_category,train_category,test_category]).T.to_csv('distribution of all,train and test.csv')

In [47]:
# useless
# df_rewards['currencies'] = df_rewards.pledges.apply(lambda x: set([p['money'][0] for p in x]))
# df_rewards[df_rewards['currencies'].apply(len)==0]